<a href="https://colab.research.google.com/github/oldaandozerskaya/covid_news/blob/main/aaai_ensembling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
input = 'Breathlessness excessive fatigue and muscle aches from COVID can last for months. https://t.co/OUhBRirKpE'

path = '/content/drive/My Drive/covid_fake_news/ct_model'

#import libs

In [ ]:
import numpy as np

!pip install transformers
import transformers
from transformers import BertForSequenceClassification, BertTokenizer

import torch
from torch.utils.data import TensorDataset, SequentialSampler, DataLoader

!pip install tweet-preprocessor
import preprocessor as pp

pp.set_options(pp.OPT.URL)

!pip install emoji
import emoji

#text preprocessing

In [ ]:
text = pp.tokenize(input)
text = emoji.demojize(text)
text = text.lower()

In [ ]:
model_name = 'digitalepidemiologylab/covid-twitter-bert'
tokenizer = BertTokenizer.from_pretrained(model_name)

input_ids = []
attention_masks = []
encoded_dict = tokenizer.encode_plus(
                        text,                    
                        truncation = True,
                        add_special_tokens = True, 
                        max_length = 128,           
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',    
                   )
input_ids.append(encoded_dict['input_ids'])
attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


#load models and predict



In [ ]:
prediction = []

for i in range(1,4):
  pytorch_model = BertForSequenceClassification.from_pretrained(path + str(i) + '/', from_tf=False)
  pytorch_model.eval()
  output = pytorch_model(input_ids, attention_masks)[0].detach().cpu().numpy()
  prediction.append(output)

del(pytorch_model)

In [ ]:
flat_prediction = [item for sublist in prediction for item in sublist]
flat_prediction = np.argmax(flat_prediction, axis=1).flatten()
result = 'fake' if np.sum(flat_prediction)>1 else 'real'
result

'real'